In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 1}) #device_count = {'CPU' : 1, 'GPU' : 1}
config.gpu_options.allow_growth = True #dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (onn which device the operation ran)

sess = tf.Session(config=config)
set_session(sess) # set this Tensorflow session as the default session for Keras

In [3]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

#conv_base.summary()

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [5]:
print('This is the number of trainable weights before freezing the conv base: ', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base:  30


In [6]:
conv_base.trainable = False
  
print('This is the number of trainable weights after freezing the conv base: ', len(model.trainable_weights))
model.summary()

This is the number of trainable weights after freezing the conv base:  4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

base_dir = '/workspace/dataset/MyExamples/dogs-vs-cats-original/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150), 
    batch_size=20, 
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150), 
    batch_size=20, 
    class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

import datetime, time
start_time = datetime.datetime.now()
#start_time = time.asctime(time.localtime(time.time()))
print("Start Time: ", start_time)

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=30, 
                              validation_data=validation_generator, validation_steps=50)

complete_time = datetime.datetime.now()
print("Complete Time: ", complete_time)
print ("Running Time: ", (complete_time - start_time).seconds, "seconds")

Start Time:  2019-07-03 03:13:01.997651
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
  5/100 [>.............................] - ETA: 2:06 - loss: 0.7641 - acc: 0.5200

KeyboardInterrupt: 

In [ ]:
model.save('cats_and_dogs_small_1.h5')

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

#print('This is the number of trainable weights after freezing the conv base: ', len(model.trainable_weights))

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

import datetime, time
start_time = datetime.datetime.now()
#start_time = time.asctime(time.localtime(time.time()))
print("Start Time: ", start_time)

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=30, 
                              validation_data=validation_generator, validation_steps=50)

complete_time = datetime.datetime.now()
print("Complete Time: ", complete_time)
print ("Running Time: ", (complete_time - start_time).seconds, "seconds")

In [ ]:
model.save('cats_and_dogs_vgg1_fine_tuning_1.h5')
model.save_weights('cats_and_dogs_vgg1_fine_tuning_weights_1.h5')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()